## Scaling 종류 별 예측성능 검정 함수

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso, ElasticNet, Ridge
%matplotlib inline

# boston 데이터 세트 로드
boston = load_boston()

# boston 데이터 세트 DataFrame 변환
bostonDF = pd.DataFrame(boston.data, columns = boston.feature_names)

# boston 데이터 세트의 target 배열은 주택 가격임. 이를 PRICE 칼럼으로 DataFrame에 추가함.
bostonDF['PRICE'] = boston.target
print("Boston 데이터 세트 크기 :", bostonDF.shape)
bostonDF.head()

Boston 데이터 세트 크기 : (506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

y_target = bostonDF['PRICE']
X_data = bostonDF.drop(['PRICE'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.3, random_state = 37)

## get_linear_reg_eval 함수

In [8]:
# alpha 값에 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DataFrame으로 반환
def get_linear_reg_eval(model_name, params=None, X_data_n = None, y_target_n=None, verbose=True):
    coeff_df = pd.DataFrame()
    if verbose : print("####### ", model_name, "#######")
    for param in params:
        if model_name == 'Ridge':
            model = Ridge(alpha=param)
        elif model_name == 'Lasso':
            model = Lasso(alpha=param)
        elif model_name =='ElasticNet':
            model = ElasticNet(alpha=param, l1_ratio=0.7)
        
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n, scoring='neg_mean_squared_error', cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print("alpha {0} 일 때 5 folds 의 평균 RMSE : {1:.3f} ".format(param,avg_rmse))
        
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        model.fit(X_data, y_target)
        
        #alpha에 따른 피처별 회귀 계수를 Series로 반환하고 이를 DataFrame의 칼럼으로 추가.
        coeff = pd.Series(data=model.coef_, index=X_data.columns)
        colname='alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df
# end of get_linear_reg_eval        

In [14]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# method는 표준 정규 분포 변환(Standard), 최댓값/최솟값 정규화(MinMax), 로그변환(Log) 결정
# p_degree는 다항식 특성을 추가할 때 적용. p_degree는 2 이상 부여하지 않음.

def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data
        
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
    
    return scaled_data

## Ridge 클래스의 alpha값을 변화시키면서 RMSE확인

In [15]:
# Ridge의 alpha값을 다르게 적용하고 다양한 데이터 변환 방법에 따른 RMSE 추출.
alphas = [0.1, 1, 10, 100]

# 5개 방식으로 변환. 먼저 원본 그대로, 표준정규 분포, 표준정규 분포 + 다항식 특성
# 최대/최소 정규화, 최대/최소 정규화 + 다항식 특성, 로그변환
scale_methods = [(None,None), ('Standard',None), ('Standard', 2), ('MinMax', None), ('MinMax', 2), ('Log', None)]

for scale_method in scale_methods:
    X_data_scaled = get_scaled_data(method=scale_method[0], p_degree=scale_method[1], input_data=X_data)
    print("\n## 변환 유형:{0}, Polynomial Degree:{1}".format(scale_method[0], scale_method[1]))
    
    get_linear_reg_eval('Ridge', params=alphas, X_data_n = X_data_scaled, y_target_n = y_target, verbose=False)


## 변환 유형:None, Polynomial Degree:None
alpha 0.1 일 때 5 folds 의 평균 RMSE : 5.788 
alpha 1 일 때 5 folds 의 평균 RMSE : 5.653 
alpha 10 일 때 5 folds 의 평균 RMSE : 5.518 
alpha 100 일 때 5 folds 의 평균 RMSE : 5.330 

## 변환 유형:Standard, Polynomial Degree:None
alpha 0.1 일 때 5 folds 의 평균 RMSE : 5.826 
alpha 1 일 때 5 folds 의 평균 RMSE : 5.803 
alpha 10 일 때 5 folds 의 평균 RMSE : 5.637 
alpha 100 일 때 5 folds 의 평균 RMSE : 5.421 

## 변환 유형:Standard, Polynomial Degree:2
alpha 0.1 일 때 5 folds 의 평균 RMSE : 8.827 
alpha 1 일 때 5 folds 의 평균 RMSE : 6.871 
alpha 10 일 때 5 folds 의 평균 RMSE : 5.485 
alpha 100 일 때 5 folds 의 평균 RMSE : 4.634 

## 변환 유형:MinMax, Polynomial Degree:None
alpha 0.1 일 때 5 folds 의 평균 RMSE : 5.764 
alpha 1 일 때 5 folds 의 평균 RMSE : 5.465 
alpha 10 일 때 5 folds 의 평균 RMSE : 5.754 
alpha 100 일 때 5 folds 의 평균 RMSE : 7.635 

## 변환 유형:MinMax, Polynomial Degree:2
alpha 0.1 일 때 5 folds 의 평균 RMSE : 5.298 
alpha 1 일 때 5 folds 의 평균 RMSE : 4.323 
alpha 10 일 때 5 folds 의 평균 RMSE : 5.185 
alpha 100 일 때 5 folds 의 평균 RMSE : 6